In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from numpy import *
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# 第1、2题结果

In [ ]:
# 获取候选1项集，dataSet为事务集。返回一个list，每个元素都是set集合
def createC1(dataSet):
    C1 = []   # 元素个数为1的项集（非频繁项集，因为还没有同最小支持度比较）
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
    C1.sort()  # 这里排序是为了，生成新的候选集时可以直接认为两个n项候选集前面的部分相同
    # 因为除了候选1项集外其他的候选n项集都是以二维列表的形式存在，所以要将候选1项集的每一个元素都转化为一个单独的集合。
    return list(map(frozenset, C1))   #map(frozenset, C1)的语义是将C1由Python列表转换为不变集合（frozenset，Python中的数据结构）

In [ ]:
# 找出候选集中的频繁项集
# dataSet为全部数据集，Ck为大小为k（包含k个元素）的候选项集，minSupport为设定的最小支持度
def scanD(dataSet, Ck, minSupport):
    ssCnt = {}   # 记录每个候选项的个数
    for tid in dataSet:
        for can in Ck:
            if can.issubset(tid):
                ssCnt[can] = ssCnt.get(can, 0) + 1   # 计算每一个项集出现的频率
    numItems = float(len(dataSet))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key] / numItems
        if support >= minSupport:
            retList.insert(0, key)  #将频繁项集插入返回列表的首部
        supportData[key] = support
    return retList, supportData   #retList为在Ck中找出的频繁项集（支持度大于minSupport的），supportData记录各频繁项集的支持度

In [ ]:
# 通过频繁项集列表Lk和项集个数k生成候选项集C(k+1)。
def aprioriGen(Lk, k):
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i + 1, lenLk):
            # 前k-1项相同时，才将两个集合合并，合并后才能生成k+1项
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]   # 取出两个集合的前k-1个元素
            L1.sort(); L2.sort()
            if L1 == L2:
                retList.append(Lk[i] | Lk[j])
    return retList

In [ ]:
# 获取事务集中的所有的频繁项集
# Ck表示项数为k的候选项集，最初的C1通过createC1()函数生成。Lk表示项数为k的频繁项集，supK为其支持度，Lk和supK由scanD()函数通过Ck计算而来。
def apriori(dataSet, minSupport):
    C1 = createC1(dataSet)  # 从事务集中获取候选1项集
    D = list(map(set, dataSet))  # 将事务集的每个元素转化为集合
    L1, supportData = scanD(D, C1, minSupport)  # 获取频繁1项集和对应的支持度
    L = [L1]  # L用来存储所有的频繁项集
    k = 2
    while (len(L[k-2]) > 0): # 一直迭代到项集数目过大而在事务集中不存在这种n项集
        Ck = aprioriGen(L[k-2], k)   # 根据频繁项集生成新的候选项集。Ck表示项数为k的候选项集
        Lk, supK = scanD(D, Ck, minSupport)  # Lk表示项数为k的频繁项集，supK为其支持度
        L.append(Lk);supportData.update(supK)  # 添加新频繁项集和他们的支持度
        k += 1
    return L, supportData

In [ ]:
df = pd.read_csv("../input/weathernominal/weather.nominal.csv")
df.head()

In [ ]:
# df['outlook'].unique()

In [ ]:
# df['temperature'].unique()

In [ ]:
# df['humidity'].unique()

In [ ]:
# df['windy'].unique()

In [ ]:
# df['play'].unique()

In [ ]:
outlook_mapping = {
           'sunny': 10,
           'overcast': 11,
           'rainy': 12}
df['outlook'] = df['outlook'].map(outlook_mapping)

In [ ]:
# df.head()

In [ ]:
temperature_mapping = {
           'hot': 20,
           'mild': 21,
           'cool': 22}
df['temperature'] = df['temperature'].map(temperature_mapping)

In [ ]:
# df.head()

In [ ]:
humidity_mapping = {
           'high': 30,
           'normal': 31}
df['humidity'] = df['humidity'].map(humidity_mapping)

In [ ]:
# df.head()

In [ ]:
list1 = ['windy','play']
for i in list1:
    df[i]=LabelEncoder().fit_transform(df[i]).astype(np.int8)
    df[i]=df[i].astype('category')

In [ ]:
df.head()

In [ ]:
dataSet = df.values.tolist()  # 获取事务集。每个元素都是列表
# C1 = createC1(dataSet)  # 获取候选1项集。每个元素都是集合
# D = list(map(set, dataSet))  # 转化事务集的形式，每个元素都转化为集合。
# L1, suppDat = scanD(D, C1, 0.5)
# print(L1,suppDat)

L, suppData = apriori(dataSet,minSupport=0.5)
print(L)

In [ ]:
print(suppData)

# 第3题

In [ ]:
df = pd.read_csv("../input/apriori/Transactions.csv")
df.head()

In [ ]:
df.info()

In [ ]:
dataSet = df.values.tolist()

L, suppData = apriori(dataSet,minSupport=0.5)
print(L)

In [ ]:
print(suppData)